In [ ]:
import csv
from datetime import datetime


######################################################
########################### INPUT/OUTPUT CSV CONVERTER
######################################################

input_file = 'BBDi.csv'
output_file = 'BBD.csv'


################################




def parse_italian_float(num_str):
    """
    Converte stringhe tipo '198,15' in float 198.15
    """
    # Rimuove eventuali virgolette e converte la virgola in punto
    num_str = num_str.replace('"', '').replace(',', '.')
    return float(num_str)

def parse_volume(vol_str):
    """
    Converte stringhe come '83,36M' in 83360000 (int).
    Gestisce anche suffissi K, M, B.
    """
    vol_str = vol_str.replace('"', '').strip()
    multiplier = 1
    if 'K' in vol_str.upper():
        multiplier = 1000
        vol_str = vol_str.upper().replace('K', '')
    elif 'M' in vol_str.upper():
        multiplier = 1_000_000
        vol_str = vol_str.upper().replace('M', '')
    elif 'B' in vol_str.upper():
        multiplier = 1_000_000_000
        vol_str = vol_str.upper().replace('B', '')

    # Converte la virgola decimale in punto
    vol_str = vol_str.replace(',', '.').strip()
    # Se dopo aver rimosso M/K/B rimane vuoto, significa volume 0
    if not vol_str:
        return 0

    return int(float(vol_str) * multiplier)

def parse_italian_date(date_str):
    """
    Converte 'gg.mm.aaaa' in 'aaaa-mm-gg'.
    """
    dt = datetime.strptime(date_str, '%d.%m.%Y')
    return dt.strftime('%Y-%m-%d')



with open(input_file, 'r', encoding='utf-8') as fin, \
     open(output_file, 'w', encoding='utf-8', newline='') as fout:

    reader = csv.reader(fin, delimiter=',')
    writer = csv.writer(fout, delimiter=',')

    # Scriviamo la nuova intestazione (tipo Yahoo Finance)
    writer.writerow(["Date","Open","High","Low","Close","Volume","Adj Close"])

    # Leggiamo e scartiamo l’intestazione vecchia
    old_header = next(reader, None)

    for row in reader:
        # Nel tuo CSV italiano, l'ordine è:
        #  0) Data
        #  1) Ultimo
        #  2) Apertura
        #  3) Massimo
        #  4) Minimo
        #  5) Vol.
        #  6) Var. %

        it_date      = row[0]
        it_ultimo    = row[1]  # Close
        it_apertura  = row[2]  # Open
        it_massimo   = row[3]  # High
        it_minimo    = row[4]  # Low
        it_volume    = row[5]
        it_var       = row[6]  # Adj Close (qui usiamo la variazione %)

        en_date  = parse_italian_date(it_date)
        en_open  = parse_italian_float(it_apertura)
        en_high  = parse_italian_float(it_massimo)
        en_low   = parse_italian_float(it_minimo)
        en_close = parse_italian_float(it_ultimo)
        en_vol   = parse_volume(it_volume)

        # Variazione % come “Adj Close”
        var_str = it_var.replace('%', '').strip()  # "4,06%" -> "4,06"
        en_adj_close = parse_italian_float(var_str)  # 4.06

        writer.writerow([en_date, en_open, en_high, en_low, en_close, en_vol, en_adj_close])


FileNotFoundError: [Errno 2] No such file or directory: 'BBDi.csv'